In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pywt

In [2]:
df = pd.read_excel("df_20.xlsx")
df.head()

,AvgAir_T_TempC,MaxAir_T_TempC,MinAir_T_TempC,Avg_Soil_TP20_TempC,Max_Soil_TP20_TempC,Min_Soil_TP20_TempC
0,8.16,11.96,1.40,8.530,8.85,8.230
1,5.48,7.33,3.47,7.926,8.28,7.774
2,6.41,9.25,1.81,8.130,8.47,7.927
3,9.17,10.67,5.04,8.790,9.15,8.440
4,3.38,5.71,-0.33,8.590,9.09,8.280


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038 entries, 0 to 1037
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AvgAir_T_TempC       1038 non-null   float64
 1   MaxAir_T_TempC       1038 non-null   float64
 2   MinAir_T_TempC       1038 non-null   float64
 3   Avg_Soil_TP20_TempC  1038 non-null   float64
 4   Max_Soil_TP20_TempC  1038 non-null   float64
 5   Min_Soil_TP20_TempC  1038 non-null   float64
dtypes: float64(6)
memory usage: 48.8 KB


In [4]:
df["Avg_Soil_TP20_TempC_Plus_1"] = df["Avg_Soil_TP20_TempC"].shift(-1)
df["Avg_Soil_TP20_TempC_Plus_2"] = df["Avg_Soil_TP20_TempC"].shift(-2)

df["Max_Soil_TP20_TempC_Plus_1"] = df["Max_Soil_TP20_TempC"].shift(-1)
df["Max_Soil_TP20_TempC_Plus_2"] = df["Max_Soil_TP20_TempC"].shift(-2)

df["Min_Soil_TP20_TempC_Plus_1"] = df["Min_Soil_TP20_TempC"].shift(-1)
df["Min_Soil_TP20_TempC_Plus_2"] = df["Min_Soil_TP20_TempC"].shift(-2)

df["AvgAir_T_TempC_Minus_6"] = df["AvgAir_T_TempC"].shift(6)
df["AvgAir_T_TempC_Minus_5"] = df["AvgAir_T_TempC"].shift(5)
df["AvgAir_T_TempC_Minus_4"] = df["AvgAir_T_TempC"].shift(4)
df["AvgAir_T_TempC_Minus_3"] = df["AvgAir_T_TempC"].shift(3)
df["AvgAir_T_TempC_Minus_2"] = df["AvgAir_T_TempC"].shift(2)
df["AvgAir_T_TempC_Minus_1"] = df["AvgAir_T_TempC"].shift(1)

df["MaxAir_T_TempC_Minus_6"] = df["MaxAir_T_TempC"].shift(6)
df["MaxAir_T_TempC_Minus_5"] = df["MaxAir_T_TempC"].shift(5)
df["MaxAir_T_TempC_Minus_4"] = df["MaxAir_T_TempC"].shift(4)
df["MaxAir_T_TempC_Minus_3"] = df["MaxAir_T_TempC"].shift(3)
df["MaxAir_T_TempC_Minus_2"] = df["MaxAir_T_TempC"].shift(2)
df["MaxAir_T_TempC_Minus_1"] = df["MaxAir_T_TempC"].shift(1)


df["MinAir_T_TempC_Minus_6"] = df["MinAir_T_TempC"].shift(6)
df["MinAir_T_TempC_Minus_5"] = df["MinAir_T_TempC"].shift(5)
df["MinAir_T_TempC_Minus_4"] = df["MinAir_T_TempC"].shift(4)
df["MinAir_T_TempC_Minus_3"] = df["MinAir_T_TempC"].shift(3)
df["MinAir_T_TempC_Minus_2"] = df["MinAir_T_TempC"].shift(2)
df["MinAir_T_TempC_Minus_1"] = df["MinAir_T_TempC"].shift(1)

In [5]:
df = df[["AvgAir_T_TempC_Minus_6","MaxAir_T_TempC_Minus_6","MinAir_T_TempC_Minus_6",
          "AvgAir_T_TempC_Minus_5","MaxAir_T_TempC_Minus_5","MinAir_T_TempC_Minus_5",
          "AvgAir_T_TempC_Minus_4","MaxAir_T_TempC_Minus_4","MinAir_T_TempC_Minus_4",
          "AvgAir_T_TempC_Minus_3","MaxAir_T_TempC_Minus_3","MinAir_T_TempC_Minus_3",
          "AvgAir_T_TempC_Minus_2","MaxAir_T_TempC_Minus_2","MinAir_T_TempC_Minus_2",
          "AvgAir_T_TempC_Minus_1","MaxAir_T_TempC_Minus_1","MinAir_T_TempC_Minus_1",
          "AvgAir_T_TempC","MaxAir_T_TempC","MinAir_T_TempC",
          "Avg_Soil_TP20_TempC","Max_Soil_TP20_TempC","Min_Soil_TP20_TempC",
          "Avg_Soil_TP20_TempC_Plus_1","Max_Soil_TP20_TempC_Plus_1","Min_Soil_TP20_TempC_Plus_1",
          "Avg_Soil_TP20_TempC_Plus_2","Max_Soil_TP20_TempC_Plus_2","Min_Soil_TP20_TempC_Plus_2"]]
df.dropna(inplace=True)

In [13]:
df.reset_index(drop="True", inplace=True)

In [7]:
def WaveletTransform(x,walvelettype,level):
    
    size = np.shape(x)
    length = (size[0] // (2**level)) * (2 ** level)
    x = x.iloc[0:length]
    bigmats = []
    for i in x.columns:
        vec = x[i].values
        cAs_cDs = pywt.swt(vec, wavelet=walvelettype, level=level,axis=0) 
        for j in cAs_cDs:
            bigmats.append(j[0])
            bigmats.append(j[1])

    
    features_num = len(bigmats)
    x = np.zeros((length, features_num))
    for i in range(0 ,len(bigmats)):
        for j in range(0,len(bigmats[0])):
            x[j,i] = x[j,i] + bigmats[i][j]
    
    return(x)

In [8]:
x = df.drop(["Avg_Soil_TP20_TempC","Max_Soil_TP20_TempC","Min_Soil_TP20_TempC",
            "Avg_Soil_TP20_TempC_Plus_1","Max_Soil_TP20_TempC_Plus_1","Min_Soil_TP20_TempC_Plus_1",
            "Avg_Soil_TP20_TempC_Plus_2","Max_Soil_TP20_TempC_Plus_2","Min_Soil_TP20_TempC_Plus_2"],axis=1)
x

,AvgAir_T_TempC_Minus_6,MaxAir_T_TempC_Minus_6,MinAir_T_TempC_Minus_6,AvgAir_T_TempC_Minus_5,MaxAir_T_TempC_Minus_5,MinAir_T_TempC_Minus_5,AvgAir_T_TempC_Minus_4,MaxAir_T_TempC_Minus_4,MinAir_T_TempC_Minus_4,AvgAir_T_TempC_Minus_3,...,MinAir_T_TempC_Minus_3,AvgAir_T_TempC_Minus_2,MaxAir_T_TempC_Minus_2,MinAir_T_TempC_Minus_2,AvgAir_T_TempC_Minus_1,MaxAir_T_TempC_Minus_1,MinAir_T_TempC_Minus_1,AvgAir_T_TempC,MaxAir_T_TempC,MinAir_T_TempC
6,8.16,11.96,1.40,5.48,7.33,3.47,6.41,9.25,1.81,9.17,...,5.04,3.38,5.71,-0.33,3.29,8.13,-1.95,5.03,5.82,4.16
7,5.48,7.33,3.47,6.41,9.25,1.81,9.17,10.67,5.04,3.38,...,-0.33,3.29,8.13,-1.95,5.03,5.82,4.16,4.78,6.71,2.22
8,6.41,9.25,1.81,9.17,10.67,5.04,3.38,5.71,-0.33,3.29,...,-1.95,5.03,5.82,4.16,4.78,6.71,2.22,2.86,5.72,0.77
9,9.17,10.67,5.04,3.38,5.71,-0.33,3.29,8.13,-1.95,5.03,...,4.16,4.78,6.71,2.22,2.86,5.72,0.77,2.60,5.80,-0.23
10,3.38,5.71,-0.33,3.29,8.13,-1.95,5.03,5.82,4.16,4.78,...,2.22,2.86,5.72,0.77,2.60,5.80,-0.23,6.83,14.17,-0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,19.75,29.30,9.16,18.19,24.56,10.54,15.45,24.52,4.61,19.27,...,9.93,16.55,21.89,10.94,14.50,20.54,6.70,15.02,23.63,5.25
1032,18.19,24.56,10.54,15.45,24.52,4.61,19.27,26.70,9.93,16.55,...,10.94,14.50,20.54,6.70,15.02,23.63,5.25,18.12,25.20,9.71
1033,15.45,24.52,4.61,19.27,26.70,9.93,16.55,21.89,10.94,14.50,...,6.70,15.02,23.63,5.25,18.12,25.20,9.71,22.42,28.65,16.47
1034,19.27,26.70,9.93,16.55,21.89,10.94,14.50,20.54,6.70,15.02,...,5.25,18.12,25.20,9.71,22.42,28.65,16.47,18.28,21.68,15.88


In [9]:
y = df[["Avg_Soil_TP20_TempC","Max_Soil_TP20_TempC","Min_Soil_TP20_TempC",
            "Avg_Soil_TP20_TempC_Plus_1","Max_Soil_TP20_TempC_Plus_1","Min_Soil_TP20_TempC_Plus_1",
            "Avg_Soil_TP20_TempC_Plus_2","Max_Soil_TP20_TempC_Plus_2","Min_Soil_TP20_TempC_Plus_2"]].values
y

array([[ 7.685,  7.827,  7.598, ...,  7.693,  8.63 ,  7.44 ],
       [10.93 , 25.42 ,  7.666, ...,  7.224,  7.451,  6.96 ],
       [ 7.693,  8.63 ,  7.44 , ...,  7.327,  7.898,  6.849],
       ...,
       [14.94 , 15.47 , 14.75 , ..., 15.5  , 15.69 , 15.42 ],
       [15.41 , 15.52 , 15.28 , ..., 15.25 , 15.68 , 14.77 ],
       [15.5  , 15.69 , 15.42 , ..., 14.33 , 14.77 , 14.11 ]])

In [12]:
x = WaveletTransform(x,"db5",5)